# 백만 개의 샘플을 가진 훈련 세트에서 (규제 없이) 훈련시킨 트리의 깊이는 대략 얼마일 것인가?

# 한 노드의 지니 불순도가 보통 그 부모 노드보다 작을 것인가?클 것인가? 일반적으로 작거나 클 것인가, 아니면 항상 작거나 클 것인가?

# 결정 트리가 훈련 세트에 과대적합되었다면 max_depth를 줄이는 것이 좋은가?

# 결정 트리가 훈련 세트에 과소적합되었다면 입력 특성의 스케일을 조정하는 것이 좋은가?

# 백만 개의 샘플을 가진 훈련 세트에 결정 트리를 훈련시키는 데 한 시간이 걸렸다면 천만 개의 샘플을 가진 훈련 세트에 결정트리를 훈련시키는 데는 대략 얼마나 걸릴 것인가?

# 십만 개의 샘플을 가진 훈련 세트가 있다면 presort = True로 지정하는 것이 훈련 속도를 높일 것인가?

# 다음 단계를 따라 moons 데이터셋에 결정 트리를 훈련시키고 세밀하게 튜닝
a. make_moons(n_samples = 10000, noise = 0.4)를 사용해 데이터셋을 생성

In [14]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples = 10000, noise = 0.4, random_state = 42)

b. 이를 train_test_split()을 사용해 훈련 세트와 테스트 세트로 분리

In [15]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

c. DecisionTreeClassifier의 최적의 매개변수를 찾기 위해 교차 검증과 함께 그리드 탐색을 수행(GridSearchCV를 사용)
- 여러 가지 max_leaf_nodes 값 사용

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'max_leaf_nodes' : list(range(2,100)), 'min_samples_split' : [2,3,4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, verbose = 1, cv = 3)

grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 882 out of 882 | elapsed:   14.6s finished


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [17]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=17, random_state=42)

d. 찾은 매개변수를 사용해 전체 훈련 세트에 대해 모델을 훈련시키고 테스트 세트에서 성능 측정. 대략 85~87%의 정확도

In [18]:
from sklearn.metrics import accuracy_score 

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

# 다음 단계를 따라 랜덤 포레스트 생성
a. 훈련 세트의 서브셋을 1000개 생성. 각각은 무작위로 선택된 100개의 샘플을 포함(사이킷런의 ShuffleSplit)을 사용

In [21]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits = n_trees, test_size = len(X_train) - n_instances, random_state = 42)
for mini_train_index, mini_test_index in rs.split(X_train) :
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

b. 7번 문제에서의 최적의 매개변수를 사용해 각 서브셋에 결정 트리를 훈련. 테스트 세트로 이 1000개의 결정 트리를 평가. 더 작은 데이터셋에서 훈련되었기 때문에 이 결정 트리는 앞서 만든 결정 트리보다 성능이 약간 떨어져 약 80% 정도의 정확도를 낸다

In [22]:
from sklearn.base import clone 

forest = [clone(grid_search_cv. best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets) :
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    
np.mean(accuracy_scores)

0.8054499999999999

c. 각 테스트 세트 샘플에 대해 1000개의 결정 트리 예측을 만들고 다수로 나온 예측만 취하기(사이파이의 mode()함수 이용). 테스트 세트에 대한 다수결 예측이 만들어진다

In [24]:
Y_pred = np.empty([n_trees, len(X_test)], dtype = np.uint8)

for tree_index, tree in enumerate(forest) : 
    Y_pred[tree_index] = tree.predict(X_test)

In [25]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis = 0)

d. 테스트 세트에서 이 예측을 평가

In [26]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872